# Main Feedback Generation Notebook
This notebook handles preprocessing, model interaction, and feedback generation.

**--Set up: Github, Paths, Imports**

In [1]:
# Mount Google Drive (optional, you'll get a prompt to authorize account)
# from google.colab import drive
# drive.mount('/content/drive')

# Start in root Colab directory to avoid nesting
%cd /content

# Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/ML-name/project.git
%cd project

# List all branches (optional, for checking)
!git branch -a

/content
Cloning into 'project'...
remote: Enumerating objects: 627, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 627 (delta 105), reused 102 (delta 59), pack-reused 430 (from 1)
Receiving objects: 100% (627/627), 1.42 MiB | 5.87 MiB/s, done.
Resolving deltas: 100% (295/295), done.
/content/project
* main
  remotes/origin/16-steps-for-phase-1
  remotes/origin/HEAD -> origin/main
  remotes/origin/feat/create-gradio-ui
  remotes/origin/feat/preprocess-the-data-into-a-normalized-format
  remotes/origin/fix/imports-and-file-paths
  remotes/origin/main
  remotes/origin/prompting/Deepseek
  remotes/origin/prompting/Gemini
  remotes/origin/refactor/docs-and-APIs
  remotes/origin/refactor/py-and-output-documents
  remotes/origin/scraping/body
  remotes/origin/scraping/inline_feedback
  remotes/origin/scraping/rubric_table
  remotes/origin/wip/prompting


In [2]:
# Checkout YOUR branch (!!replace "your-branch-name"!!)
!git checkout scraping/inline_feedback

Branch 'scraping/inline_feedback' set up to track remote branch 'scraping/inline_feedback' from 'origin'.
Switched to a new branch 'scraping/inline_feedback'


**--Install required libraries and set paths**

In [3]:
%pip install -r requirements.txt
!pip install python-docx

# import data path for loading files
import os

# Base data directory
data_base = '/content/project/data'

# Paths to specific subfolders
raw_data_path = os.path.join(data_base, 'raw')
processed_data_path = os.path.join(data_base, 'processed')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.7 MB/s eta 0:00:00


In [16]:
%%writefile /content/project/tropos/__init__.py

# This file marks tropos as a Python package.

#from .models.gpt import generate_feedback
#from .models.trained import load_model

from .preprocess_docx.rubric import parse_rubric
from .preprocess_docx.submission import parse_submission
from .preprocess_docx.requirements import parse_requirements
#from .preprocess_docx.comments import parse_comments
#from .preprocess_docx import StudentSubmission
from .preprocess_docx.inline_feedback import CommentExtractor, extract_comments

# Optional UI export
#from .gradio.ui import launch_ui

#from spire.doc import *
#from spire.doc.common import *



# Starts the program
def main():
    # Tests the ui
    # make_ui()

    # Tests the docx data extraction

    requirements_doc = Document()
    requirements_doc.LoadFromFile(".data/raw/Requirements.docx")

    submission_doc = Document()
    submission_doc.LoadFromFile("data/raw/Student 1/Student 1 Part 1.docx")

    print("Testing StudentSubmission class")
    StudentSubmission(submission_doc, requirements_doc)



Overwriting /content/project/tropos/__init__.py


In [14]:
%%writefile /content/project/tropos/preprocess_docx/inline_feedback.py
from docx import Document
from docx.opc.constants import RELATIONSHIP_TYPE as RT
from docx.oxml import parse_xml
from docx.oxml.ns import qn
import zipfile

class CommentExtractor:
    def __init__(self, doc_path):
        self.doc_path = doc_path
        self.doc = Document(doc_path)
        self.comments = []
        self.comment_refs = {}

    def extract_comments(self):
        """Main method to extract all comments and their context"""
        self._extract_comment_content()
        self._find_comment_references()
        return self

    def _extract_comment_content(self):
        """Extract comment content from the docx archive"""
        try:
            with zipfile.ZipFile(self.doc_path) as z:
                with z.open('word/comments.xml') as f:
                    comments_xml = parse_xml(f.read())
                    namespace = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}
                    for comment in comments_xml.xpath('//w:comment', namespaces=namespace):
                        self.comments.append({
                            'id': comment.get(qn('w:id')),
                            'author': comment.get(qn('w:author'), 'Unknown'),
                            'date': comment.get(qn('w:date'), ''),
                            'text': ''.join([
                                node.text for node in
                                comment.xpath('.//w:t', namespaces=namespace)
                                if node.text
                            ]).strip()
                        })
        except KeyError:
            print("Document contains no comments")
        except Exception as e:
            print(f"Error reading comments: {e}")

    def _find_comment_references(self):
        """Find comment references in document text"""
        for paragraph in self.doc.paragraphs:
            for run in paragraph.runs:
                if hasattr(run, '_element'):
                    if comment_refs := run._element.xpath('.//w:commentReference'):
                        comment_id = comment_refs[0].get(qn('w:id'))
                        self.comment_refs[comment_id] = {
                            'text': run.text.strip(),
                            'paragraph': paragraph.text.strip()
                        }

    def get_results(self):
        """Return structured comment data"""
        return [{
            'comment_id': c['id'],
            'comment_text': c['text'],
            'commented_text': self.comment_refs.get(c['id'], {}).get('text', ''),
            'paragraph': self.comment_refs.get(c['id'], {}).get('paragraph', ''),
            'author': c['author'],
            'date': c['date']
        } for c in self.comments]

# Add this if you want a simple function interface
def extract_comments(doc_path):
    return CommentExtractor(doc_path).extract_comments().get_results()


Overwriting /content/project/tropos/preprocess_docx/inline_feedback.py


**--Import modules (youre working on)**
<br>*each of our classes will be what will merge to this notebook (im p sure)*
<br>only loads what you explicitly request
<br>(this helps keep memory low and import fast)
<br> *the following is an example with my Rubric module*

## Inline Feedback Scraper

In [15]:
import os
import sys
import json
sys.path.append('/content/project')

# Import the CommentExtractor
from tropos.preprocess_docx.inline_feedback import CommentExtractor

# Path to raw student submissions
data_dir = "/content/project/data/raw/"
output_file = "/content/project/tropos/preprocess_docx/submission.py"

# Dictionary to store all comments
all_comments = {}

# Iterate through each student folder
for student_folder in os.listdir(data_dir):
    student_path = os.path.join(data_dir, student_folder)

    if os.path.isdir(student_path):  # Ensure it's a folder
        student_comments = []

        # Iterate through all .docx files in the student's folder
        for file_name in os.listdir(student_path):
            if file_name.endswith(".docx"):  # Process only Word documents
                doc_path = os.path.join(student_path, file_name)

                # Extract comments from the document
                extractor = CommentExtractor(doc_path).extract_comments()
                comments = extractor.get_results()

                # Store the extracted comments
                if comments:
                    student_comments.extend(comments)

        # Save extracted comments for the student
        if student_comments:
            all_comments[student_folder] = student_comments

# Write the results to submission.py
with open(output_file, "w") as f:
    f.write("comments_data = ")
    json.dump(all_comments, f, indent=4)

print(f"Saved extracted comments to {output_file}")


Document contains no comments
Document contains no comments
Document contains no comments
Document contains no comments
Document contains no comments
Document contains no comments
Document contains no comments
Document contains no comments
Document contains no comments
Document contains no comments
Document contains no comments
Document contains no comments
Saved extracted comments to /content/project/tropos/preprocess_docx/submission.py


In [21]:
#!git config --global user.email "connerr1978@gmail.com"
#!git config --global user.name "ConnerRgit"
#!git add .
#!git commit -m "feat: Add inline_feedback.py for DOCX comment extraction"


On branch scraping/inline_feedback
Your branch is ahead of 'origin/scraping/inline_feedback' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


fatal: could not read Username for 'https://github.com': No such device or address
